In [6]:
H = [2500, 3000, 4000, 5500, 4500, 3500]

6-element Vector{Int64}:
 2500
 3000
 4000
 5500
 4500
 3500

In [17]:
using JuMP, Cbc

m = Model()

@variable(m, x[1:6] >= 0) # of hired employees in period t
@variable(m, y[1:6] >= 0) # of retiring employees in period t
@variable(m, s[0:5] >= 0) # of carryover labor hours in from period t-1 to t
#@variable(m, a[1:6] >= 0) # of hrs an employee works in period t
@variable(m, z[0:6] >= 0) # of current employees in period t

@objective(m, Min, 5000 * sum(x) + 10000 * sum(y) + 2000 * sum(z))

@constraint(m, curr_work[t in 1:6], z[t] == z[t-1] + x[t] - y[t])
@constraint(m, start_worker, z[0] == 35)

@constraint(m, carryover[t in 1:5], s[t] == s[t-1] + (H[t] - a[t])) # ??
@constraint(m, start_carryover, s[0] == 0)

@constraint(m, max_retire, sum(y) <= 8)
@constraint(m, max_worker_hrs[t in 1:6], a[t] * 160 <= H[t])
@constraint(m, max_carryover, sum(s) <= 400)
@constraint(m, hrs_req[t in 1:6], a[t] * z[t] + s[t-1] <= H[t]) # ??
@constraint(m, tot_hrs_meet, sum(a[t] * z[t] for t in 1:6) + sum(s[i] for i in 1:5) == sum(H))

tot_hrs_meet : a[1]*z[1] + a[2]*z[2] + a[3]*z[3] + a[4]*z[4] + a[5]*z[5] + a[6]*z[6] + s[1] + s[2] + s[3] + s[4] + s[5] == 23000.0

In [18]:
set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

LoadError: MathOptInterface.UnsupportedConstraint{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.EqualTo{Float64}}: `MathOptInterface.ScalarQuadraticFunction{Float64}`-in-`MathOptInterface.EqualTo{Float64}` constraint is not supported by the model.